## Install Packages

In [1]:
!conda install -y -c conda-forge faiss-gpu
!apt-get -y update
!apt-get -y install libatlas-base-dev

done
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.14.0
  latest version: 25.7.0

Please update conda by running

    $ conda update -n base -c conda-forge conda



# All requested packages already installed.

Retrieving notices: ...working... done
Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [102 kB]       
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [102 kB]     
Get:4 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [102 kB]     
Fetched 305 kB in 1s (278 kB/s)                                                
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libatlas-base-dev is already the newest version (3.10.3-5).
0 upgraded, 0 newly installed, 0 to remove and 83 not upgraded.


## Load Data

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.random_projection import GaussianRandomProjection

from tqdm import tqdm

import faiss

In [3]:
df = pd.read_csv("story_dataset.csv")
df

,prompt_id,prompt,story,hidden_state_file,len_generated_story,len_new_story
0,1,Once upon a time there was a dragon,Once upon a time there was a dragon named Blaz...,./hidden_states/prompt_1.npz,270,271
1,1,Once upon a time there was a dragon,Once upon a time there was a dragon named Spar...,./hidden_states/prompt_1.npz,349,350
2,1,Once upon a time there was a dragon,Once upon a time there was a dragon named Scor...,./hidden_states/prompt_1.npz,278,278
3,1,Once upon a time there was a dragon,Once upon a time there was a dragon. The drago...,./hidden_states/prompt_1.npz,117,118
4,1,Once upon a time there was a dragon,Once upon a time there was a dragon. The drago...,./hidden_states/prompt_1.npz,129,130
...,...,...,...,...,...,...
9995,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,289,290
9996,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,119,119
9997,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,127,128
9998,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,441,441


In [4]:
max_story_len = max(df["len_generated_story"])
max_story_len

522

In [5]:
hidden_states_by_layer = {}
NUM_PROMPTS = 10

for prompt_id in range(1, NUM_PROMPTS + 1):
    with np.load(f'./hidden_states/prompt_{prompt_id}.npz') as loaded_data:
        for i in tqdm(range(1000)):
            curr_hidden_states = loaded_data[f"arr_{i}"][0]
#             print(curr_hidden_states.shape)
            for layer in range(1, 2):
                padded_arr = np.zeros((max_story_len, 512))
                padded_arr_len = len(curr_hidden_states[layer][0])
                
                padded_arr[:padded_arr_len] = curr_hidden_states[layer][0]
                
                padded_arr = padded_arr.flatten().astype('float32') #FAISS expects data in type float32 instead of float64 - saves memory too!
#                 print(padded_arr.shape)
                
                if(f"layer_{layer}" in hidden_states_by_layer):
                    hidden_states_by_layer[f"layer_{layer}"].append(padded_arr)
                else:
                    hidden_states_by_layer[f"layer_{layer}"] = [padded_arr]

100%|███████████████████████████████████████████████████████████████████████████████| 1000/1000 [01:55<00:00,  8.65it/s]


In [6]:
layer_hs_array = np.array(hidden_states_by_layer["layer_1"])
layer_hs_array.shape

(10000, 267264)

## Layer 1 Clustering

In [7]:
# Use original vectors for clustering - uncomment next line and comment out last two lines

dim_reduced_vecs = layer_hs_array

# random_projector = GaussianRandomProjection(random_state = 42)
# dim_reduced_vecs = random_projector.fit_transform(layer_hs_array).astype('float32')

In [8]:
dim_reduced_vecs = np.array([v / np.linalg.norm(v) for v in dim_reduced_vecs])
dim_reduced_vecs.shape

(10000, 267264)

In [9]:
# K-means Clustering

ncentroids = NUM_PROMPTS
niter = 20
verbose = True
dim = dim_reduced_vecs.shape[1]
kmeans = faiss.Kmeans(dim, ncentroids, niter = niter, verbose = verbose, gpu = True, nredo = 10, spherical = True, max_points_per_centroid = 1000)
kmeans.train(dim_reduced_vecs)

Clustering 10000 points in 267264D to 10 clusters, redo 10 times, 20 iterations
  Preprocessing in 2.24 s
Outer iteration 0 / 10
  Iteration 19 (71.19 s, search 40.40 s): objective=4460.98 imbalance=1.027 nsplit=0       
Objective improved: keep new clusters
Outer iteration 1 / 10
  Iteration 19 (142.35 s, search 80.82 s): objective=4466.46 imbalance=1.149 nsplit=0       
Objective improved: keep new clusters
Outer iteration 2 / 10
  Iteration 19 (213.47 s, search 121.24 s): objective=4461.81 imbalance=1.067 nsplit=0       
Outer iteration 3 / 10
  Iteration 19 (284.68 s, search 161.68 s): objective=4463.14 imbalance=1.051 nsplit=0       
Outer iteration 4 / 10
  Iteration 19 (355.93 s, search 202.10 s): objective=4437.67 imbalance=1.158 nsplit=0       
Outer iteration 5 / 10
  Iteration 19 (427.08 s, search 242.54 s): objective=4462.45 imbalance=1.103 nsplit=0       
Outer iteration 6 / 10
  Iteration 19 (498.14 s, search 282.97 s): objective=4468.87 imbalance=1.088 nsplit=0       
Ob

4468.8662109375

In [10]:
kmeans.centroids #cluster centers

array([[ 1.04215257e-02,  4.51111840e-03,  1.45422788e-02, ...,
        -3.54626053e-07, -2.87363832e-07,  3.02960871e-07],
       [ 1.26520488e-02,  5.47663216e-03,  1.76547747e-02, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 1.06646260e-02,  4.61634714e-03,  1.48815010e-02, ...,
        -4.57973385e-07,  5.19763148e-07,  1.74893955e-06],
       ...,
       [ 1.13931708e-02,  4.93171252e-03,  1.58981252e-02, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 1.12661235e-02,  4.87671327e-03,  1.57208461e-02, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 1.25000356e-02,  5.41083142e-03,  1.74426492e-02, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00]], dtype=float32)

In [11]:
for centroid in kmeans.centroids:
    print(np.linalg.norm(centroid))

1.0000384
1.0000174
1.0000455
1.0000157
1.0000664
1.0000522
1.0000141
1.0000523
1.0000701
1.0000294


In [12]:
kmeans.obj #inertia at each iteration

array([2600.95947266, 4296.37939453, 4364.42041016, 4393.20458984,
       4413.18847656, 4431.125     , 4439.17480469, 4443.54541016,
       4447.21142578, 4450.43408203, 4452.828125  , 4454.50439453,
       4455.73779297, 4456.91650391, 4458.07128906, 4459.01318359,
       4459.66650391, 4460.16503906, 4460.65917969, 4460.98046875,
       2745.63793945, 4270.58056641, 4360.63916016, 4398.11279297,
       4413.78417969, 4421.38232422, 4425.50146484, 4428.52832031,
       4431.78173828, 4435.91162109, 4441.32910156, 4446.82080078,
       4451.81494141, 4456.83398438, 4460.21582031, 4462.15771484,
       4463.45751953, 4464.63183594, 4465.62744141, 4466.46337891,
       2559.20825195, 4290.31738281, 4370.55566406, 4397.65478516,
       4412.54003906, 4425.16308594, 4436.33398438, 4444.99707031,
       4449.16015625, 4451.35302734, 4452.96923828, 4454.3515625 ,
       4455.55224609, 4456.5078125 , 4457.28515625, 4458.07373047,
       4458.99462891, 4460.04443359, 4461.09277344, 4461.80859

In [13]:
normalized_vecs = [v / np.linalg.norm(v) for v in dim_reduced_vecs]

In [14]:
cos_similarities = normalized_vecs @ kmeans.centroids.T
classifications = np.argmax(cos_similarities, axis=1)

In [15]:
pd.Series(classifications).value_counts()

0    1398
6    1338
7    1206
3    1172
2    1153
1     934
8     928
4     840
5     584
9     447
dtype: int64

In [16]:
pd.Series(kmeans.index.search(dim_reduced_vecs.astype(np.float32), 1)[1].flatten()).value_counts()

0    1398
6    1338
7    1206
3    1172
2    1153
1     934
8     928
4     840
5     584
9     447
dtype: int64

In [17]:
prompt_ids = df["prompt_id"]
prompt_ids = prompt_ids.to_numpy()
prompt_ids

array([ 1,  1,  1, ..., 10, 10, 10])

In [ ]:
# Get most common centroid for each 1000 points (same label)
max_centroid_per_label = [pd.Series(classifications[i * 1000:(i + 1) * 1000]).value_counts().idxmax() for i in range(10)]
max_centroid_per_label

[6, 3, 6, 0, 2, 0, 0, 2, 0, 5]

In [ ]:
# Get most common label for each point classified to a centroid (same centroid)
centroid_labels = [np.where(classifications == i)[0] for i in range(10)]
max_label_per_centroid = [pd.Series(prompt_ids[centroid_labels[i]]).value_counts().idxmax() for i in range(10)]
max_label_per_centroid

In [ ]:
max_centroids = [centroid for centroid in max_centroid_per_label for _ in range(1000)]

max_labels = [label for label in max_label_per_centroid for _ in range(1000)]

In [ ]:
np.array(max_centroids)

In [ ]:
np.array(max_labels)

In [ ]:
label_to_centroid = {idx + 1 : max_centroid_per_label[idx] for idx in range(len(max_centroid_per_label))}

centroid_to_label = {idx : max_label_per_centroid[idx] for idx in range(len(max_label_per_centroid))}

In [ ]:
label_to_centroid

In [ ]:
centroid_to_label

In [ ]:
vectorized_map = np.vectorize(centroid_to_label.get)
classifications_to_label = vectorized_map(classifications)

classifications_to_label

In [ ]:
for i in range(10):
    print(f"Prompt {i + 1} Accuracy: ", np.mean(classifications_to_label[i * 1000:(i + 1) * 1000] == (i + 1)))

In [ ]:
print(f"Overall Accuracy: ", np.mean(classifications_to_label == prompt_ids))